## Web에서 데이터 가져오기

**requests 모듈**
- 참고 사이트 : https://realpython.com/python-requests/

In [4]:
import requests

**URL 요청하기**

In [7]:
response = requests.get('https://google.com')

In [8]:
response.status_code

200

**응답 받은 페이지가 성공적이지 않을 때**

In [10]:
url = 'https://search.daum.net/searc?w=tot&q=2021%EB%85%84%EC%98%81%ED%99%94%EC%88%9C%EC%9C%84&DA=MOR&rtmaxcoll=MOR'

In [11]:
response = requests.get(url)

In [12]:
response.status_code

404

In [14]:
response = requests.get(url)
if response.status_code == 400:
    print('status ok')
else:
    print('status error', response.status_code)

status error 404


In [16]:
# requests 모듈에서 제공하는 코드(exception 발생)
response = requests.get(url)
response.raise_for_status()

HTTPError: 404 Client Error: Not Found for url: https://search.daum.net/searc?w=tot&q=2021%EB%85%84%EC%98%81%ED%99%94%EC%88%9C%EC%9C%84&DA=MOR&rtmaxcoll=MOR

In [22]:
# Exception 처리 (try 구문 이용)
try:
    response = requests.get(url)
    response.raise_for_status()
except Exception as e:
    print('request 실패\n', e)

request 실패
 404 Client Error: Not Found for url: https://search.daum.net/searc?w=tot&q=2021%EB%85%84%EC%98%81%ED%99%94%EC%88%9C%EC%9C%84&DA=MOR&rtmaxcoll=MOR


In [26]:
url = 'https://google.com'
response = requests.get(url)
response.raise_for_status()

# fd = open('mygoogle.html', 'w', encoding='utf8') # open으로 상용하면 close를 같이 사용해봐야 함
# fd.write(response.text)
# fd.close()

with open('mygoogle.html', 'w', encoding='utf8') as fd:
    fd.write(response.text)
print('저장 완료')

저장 완료


**User Agent**
- URL을 Request하는 Client 의 종류를 싣어 보냄
- 같은 URL을 Request 하더라도 모바일 디바이스와 PC가 받는 데이터가 다른 이유임
- Crawler 같은 자동화 프로그램이 서버에 접근하면 여러 문제가 생길 수 가 있으므로 차단할 필요가 있음
- 이 부분을 마치 브라우저가 요청하듯이 바꿀 수 있는 방법이 User Agent를 조정하는 것임
- Google에서 user agent string으로 검색하면
- https://www.whatismybrowser.com/detect/what-is-my-user-agent/ 사이트에서
- 현재 브라우저의 User Agent 를 표시해줌
- Client마다 서버로 보내는 정보가 다르다는 것을 확인할 수 있음


In [27]:
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36'

headers = {'User-Agent' : user_agent}
response = requests.get(url, headers)
response.raise_for_status()

with open('mygoogle2.html', 'w', encoding='utf8') as fd:
    fd.write(response.text)

print('저장 완료')

저장 완료


## **실습 1**

**네이버 웹소설에서 스크랩핑하기**
- https://novel.naver.com/webnovel/weekday

**lxml**
- 구문분석기
- 형식을 정확히 지키지 않은 HTML 코드를 분석할 때 html.parser 보다 나음
- 닫히지 않은 태그, 계층 구조가 잘못된 태그, <head>나 <body> 태그가 없는 등의 문제에서 일일이 멈추지 않고 문제를 수정
- 속도 빠름

In [29]:
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36'
novel_url = 'https://novel.naver.com/webnovel/weekday'

headers = {'User-Agent' : user_agent}
response = requests.get(novel_url, headers)
response.raise_for_status()

with open('mynovel.html', 'w', encoding='utf8') as fd:
    fd.write(response.text)

print('저장 완료')

저장 완료


In [30]:
from bs4 import BeautifulSoup

**웹소설 통합 랭킹**

In [99]:
# 강사님 답
soup = BeautifulSoup(response.text, 'lxml')

total_rank = soup.select_one('div.component_section')
ranking_lists = total_rank.select('ul.ranking_list') # 다섯개씩 묶여있는 컬럼 3개
# print(len(ranking_lists)) #3

for ranking_list in ranking_lists:
    items = ranking_list.select('li.item') 
    # print(len(items)) #5
    for item in items:
        title = item.select_one('span.title').text
        link = item.select_one('a.link')['href']
        count = item.select_one('span.count').text
        print(f'제목 : {title}')
        print(f'링크 : https://novel.naver.com{link}')
        print(f'관심 : {count[2:]}\n')



제목 : 유리 구두의 정원
링크 : https://novel.naver.com/webnovel/list?novelId=1167230
관심 : 2,868

제목 : 나는 전남편의 비서입니다
링크 : https://novel.naver.com/best/list?novelId=1183480
관심 : 1,325

제목 : 사천당가의 장녀는 가문을 지킨다
링크 : https://novel.naver.com/webnovel/list?novelId=1167223
관심 : 2,547

제목 : 내 쓰레기들이 후회하기 시작했다
링크 : https://novel.naver.com/webnovel/list?novelId=1167235
관심 : 1,874

제목 : 이제 사랑받을 수 있을까요?
링크 : https://novel.naver.com/webnovel/list?novelId=1167236
관심 : 2,126

제목 : 아내를 숨겼다
링크 : https://novel.naver.com/webnovel/list?novelId=1167227
관심 : 2,333

제목 : 나도 처음이니까
링크 : https://novel.naver.com/best/list?novelId=1163227
관심 : 1,480

제목 : 본부장님의 여대생 키우기
링크 : https://novel.naver.com/best/list?novelId=1159848
관심 : 1,054

제목 : 파고드는 순정
링크 : https://novel.naver.com/best/list?novelId=1180572
관심 : 793

제목 : 마녀 구원
링크 : https://novel.naver.com/best/list?novelId=1180185
관심 : 822

제목 : 대공가의 새아가는 조금 사나워요
링크 : https://novel.naver.com/best/list?novelId=1138846
관심 : 1,261

제목 : 내가 가진 너의 여름
링크 : https://novel.naver.com/best/li

In [32]:
# todo


제목 : 유리 구두의 정원
링크 : https://novel.naver.com/webnovel/list?novelId=1167230
관심 : 2,859

제목 : 나는 전남편의 비서입니다
링크 : https://novel.naver.com/best/list?novelId=1183480
관심 : 1,193

제목 : 사천당가의 장녀는 가문을 지킨다
링크 : https://novel.naver.com/webnovel/list?novelId=1167223
관심 : 2,526

제목 : 언니, 이번 생엔 내가 왕비야
링크 : https://novel.naver.com/webnovel/list?novelId=934373
관심 : 2만

제목 : 내 쓰레기들이 후회하기 시작했다
링크 : https://novel.naver.com/webnovel/list?novelId=1167235
관심 : 1,864

제목 : 이제 사랑받을 수 있을까요?
링크 : https://novel.naver.com/webnovel/list?novelId=1167236
관심 : 2,122

제목 : 뺏어줘서 고마워, 언니
링크 : https://novel.naver.com/best/list?novelId=1172502
관심 : 910

제목 : 착한 오빠, 나쁜 오빠
링크 : https://novel.naver.com/best/list?novelId=1159312
관심 : 1,673

제목 : 본부장님의 여대생 키우기
링크 : https://novel.naver.com/best/list?novelId=1159848
관심 : 1,048

제목 : 키하라나스의 신부
링크 : https://novel.naver.com/best/list?novelId=1162073
관심 : 1,022

제목 : 내가 가진 너의 여름
링크 : https://novel.naver.com/best/list?novelId=1178405
관심 : 628

제목 : 복수를 위해 결혼했다
링크 : https://novel.naver

**소설 `유리 구두의 정원`의 제목, 링크, 평점 가져오기**

**한 페이지 출력**

In [150]:
# 내 풀이
url = 'https://novel.naver.com/webnovel/list?novelId=1167230'
res = requests.get(url, headers={"User-Agent": user_agent}) 
soup = BeautifulSoup(res.text, 'lxml')

novel_lists = soup.select('li.volumeComment')

for novel in novel_lists:
    title = novel.select_one('div.pic>img')
    link = novel.select_one('a')
    score = novel.select_one('span.score_area em')

    print(f'제목 : {title['alt']}')
    print(f'링크 : https://novel.naver.com{link['href']}')
    print(f'평점 : {score.text}\n')

제목 : 170. 외전 - 자업자득
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=171
평점 : 10.0

제목 : 169. 외전 - 마지막 복수
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=170
평점 : 10.0

제목 : 168. 외전 - 끝없는 열등감
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=169
평점 : 10.0

제목 : 167. 외전 - 꿈
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=168
평점 : 10.0

제목 : 166. 외전 - 겨울의 온기
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=167
평점 : 10.0

제목 : 165. 외전 - 다시, 봄
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=166
평점 : 10.0

제목 : 164. 외전 - 가을 그리고 겨울
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=165
평점 : 10.0

제목 : 163. 외전 - 여름
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=164
평점 : 9.96

제목 : 162. 외전 - 봄
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=163
평점 : 10.0

제목 : 161. 에필로그
링크 : https://novel.naver.com/webnov

In [139]:
# 강사님 답
soup = BeautifulSoup(res.text, 'lxml')

items = soup.select('ul.list_type2 li.volumeComment')
len(items)

10

In [155]:
for item in items:

    span = item.select_one('div.list_info p.subj span')
    if span:
        up = span.find_next().text
        title = item.select_one('div.list_info p.subj').get_text(strip=True)
        title = title.replace(up, '')
    else:
        title = item.select_one('p.subj').get_text(strip=True)  

    link = item.select_one('a.list_item')['href']
    rate = item.select_one('p.rating em').text
    print(f'제목 : {title}')
    print(f'링크 : https://novel.naver.com{link}')
    print(f'평점 : {rate}')
    print()


제목 : 170. 외전 - 자업자득
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=171
평점 : 10.0

제목 : 169. 외전 - 마지막 복수
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=170
평점 : 10.0

제목 : 168. 외전 - 끝없는 열등감
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=169
평점 : 10.0

제목 : 167. 외전 - 꿈
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=168
평점 : 10.0

제목 : 166. 외전 - 겨울의 온기
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=167
평점 : 10.0

제목 : 165. 외전 - 다시, 봄
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=166
평점 : 10.0

제목 : 164. 외전 - 가을 그리고 겨울
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=165
평점 : 10.0

제목 : 163. 외전 - 여름
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=164
평점 : 9.96

제목 : 162. 외전 - 봄
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=163
평점 : 10.0

제목 : 161. 에필로그
링크 : https://novel.naver.com/webnov

In [37]:
# todo


제목 : 169. 외전 - 마지막 복수
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=170
평점 : 10.0

제목 : 168. 외전 - 끝없는 열등감
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=169
평점 : 10.0

제목 : 167. 외전 - 꿈
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=168
평점 : 10.0

제목 : 166. 외전 - 겨울의 온기
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=167
평점 : 10.0

제목 : 165. 외전 - 다시, 봄
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=166
평점 : 10.0

제목 : 164. 외전 - 가을 그리고 겨울
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=165
평점 : 10.0

제목 : 163. 외전 - 여름
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=164
평점 : 9.96

제목 : 162. 외전 - 봄
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=163
평점 : 10.0

제목 : 161. 에필로그
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=162
평점 : 10.0

제목 : 160. 첫사랑
링크 : https://novel.naver.com/webnovel/det

**전체 페이지 출력**

In [158]:
# 내 풀이
for i in range(1, 19):
    url = f'https://novel.naver.com/webnovel/list?novelId=1167230&page={i}' # f'https://novel.naver.com/webnovel/list?novelId=1167230&page={}'.format(i)
    res = requests.get(url, headers={"User-Agent": user_agent}) 
    soup = BeautifulSoup(res.text, 'lxml')
    
    novel_lists = soup.select('li.volumeComment')
    print(f'---page {i} --------------------------------------------------')
    for novel in novel_lists:
        title = novel.select_one('div.pic>img')
        link = novel.select_one('a')
        score = novel.select_one('span.score_area em')
        
        print(f'제목 : {title['alt']}')
        print(f'링크 : https://novel.naver.com{link['href']}')
        print(f'평점 : {score.text}\n')

---page 1 --------------------------------------------------
제목 : 170. 외전 - 자업자득
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=171
평점 : 10.0

제목 : 169. 외전 - 마지막 복수
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=170
평점 : 10.0

제목 : 168. 외전 - 끝없는 열등감
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=169
평점 : 10.0

제목 : 167. 외전 - 꿈
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=168
평점 : 10.0

제목 : 166. 외전 - 겨울의 온기
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=167
평점 : 10.0

제목 : 165. 외전 - 다시, 봄
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=166
평점 : 10.0

제목 : 164. 외전 - 가을 그리고 겨울
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=165
평점 : 10.0

제목 : 163. 외전 - 여름
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=164
평점 : 9.96

제목 : 162. 외전 - 봄
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=163


In [39]:
# todo



---page 1 --------------------------------------------------
제목 : 169. 외전 - 마지막 복수
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=170
평점 : 10.0

제목 : 168. 외전 - 끝없는 열등감
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=169
평점 : 10.0

제목 : 167. 외전 - 꿈
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=168
평점 : 10.0

제목 : 166. 외전 - 겨울의 온기
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=167
평점 : 10.0

제목 : 165. 외전 - 다시, 봄
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=166
평점 : 10.0

제목 : 164. 외전 - 가을 그리고 겨울
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=165
평점 : 10.0

제목 : 163. 외전 - 여름
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=164
평점 : 9.96

제목 : 162. 외전 - 봄
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=163
평점 : 10.0

제목 : 161. 에필로그
링크 : https://novel.naver.com/webnovel/detail?novelId=1167230&volumeNo=162
평점 :

## **실습 2**

**다음 영화순위 검색에서 스크랩핑하기**
- 썸네일 이미지 저장하기

**2024년도 이미지 링크 출력**

In [41]:
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'
headers = {'User-Agent' : user_agent}
url = "https://search.daum.net/search?w=tot&q=2024%EB%85%84%EC%98%81%ED%99%94%EC%88%9C%EC%9C%84&DA=MOR&rtmaxcoll=MOR"

response = requests.get(url, headers)
response.raise_for_status()

with open('mymovie.html', 'w', encoding='utf8') as fd:
    fd.write(response.text)

- mymovie.html이 실제 사이트 접속해서 개발자 도구로 보는것과 다름
- 자바스크립트가 동적으로 실행되서 페이지가 생성되는 경우에는 requests 모듈만으로는 부족
- Seleinum 사용해서 시도해보기